# Question 1

In [17]:
free_matrix = 
[
0 0 1 1 0 0 0 1 1 0 0 0 0;
0 1 1 0 0 0 0 0 1 1 0 0 0;
0 0 0 1 1 0 1 1 0 1 1 1 1;
0 0 0 1 1 1 1 1 1 1 1 1 0;
0 0 0 0 0 0 1 1 1 0 0 0 0;
0 1 1 0 0 0 0 0 1 1 0 0 0;
0 0 0 1 1 1 1 0 0 0 0 0 0;
1 1 0 0 0 0 0 0 0 0 1 1 1;
1 1 1 0 0 0 0 0 0 1 1 0 0;
0 0 0 0 0 0 0 1 1 0 0 0 0;
0 0 0 0 0 0 1 1 1 0 0 0 0;
1 1 0 0 0 1 1 1 1 0 0 1 1;
1 1 1 0 1 1 0 0 0 0 0 1 1;
0 1 1 1 0 0 0 0 0 0 0 0 0;
1 1 0 0 1 1 0 0 0 0 0 0 0;
]

demand = [1 1 1 1 1 1 3 1 1 1 1 1 1]
times = ["10:00", "10:20" ,"10:40", "11:00", "11:20", "11:40", "Lunch", "1:00", "1:20", "1:40" , "2:00", "2:20", "2:40"]
names = ["Manuel" , "Luca" , "Jule" , "Michael", "Malte", "Chris", "Spyros", "Mirjam", "Matt", "Florian", "Josep", "Joel" ,"Tom" , "Daniel" ,"Anne"]

15-element Array{String,1}:
 "Manuel" 
 "Luca"   
 "Jule"   
 "Michael"
 "Malte"  
 "Chris"  
 "Spyros" 
 "Mirjam" 
 "Matt"   
 "Florian"
 "Josep"  
 "Joel"   
 "Tom"    
 "Daniel" 
 "Anne"   

In [28]:
using JuMP
m = Model()
@variable(m, x[1:15,1:13] >= 0)
for i in 1:15
    @constraint(m , dot(x[i,:], free_matrix[i,:]) == 1)
end
for i in 1:13
    @constraint(m, sum(x[:,i]) == demand[i])
end
solve(m)

:Optimal

### Final Schedule

In [29]:
x_val = getvalue(x)
for t in 1:13
    print(times[t], ": ")
    for i in 1:15
        if x_val[i, t] > 0
            print(names[i] , " ")
        end
    end
    println()
end

10:00: Joel 
10:20: Chris 
10:40: Tom 
11:00: Daniel 
11:20: Anne 
11:40: Spyros 
Lunch: Michael Malte Josep 
1:00: Manuel 
1:20: Florian 
1:40: Luca 
2:00: Matt 
2:20: Mirjam 
2:40: Jule 


# Question 2

In [38]:
pos = [
    0 20 18 30 35 33 5 5 11 2;
    0 20 10 12 0 25 27 10 0 15
]
required = [10 6 8 11 9 7 15 7 9 12]
present = [8 13 4 8 12 2 14 11 15 7]
distances = zeros( 10,10)
for i in 1:10
    for j in 1:10
        delta = pos[:,i] - pos[:,j]
        distances[i,j] = dot(delta,delta)
    end
end


10×10 Array{Float64,2}:
    0.0  800.0  424.0  1044.0  1225.0  1714.0   754.0   125.0   121.0   229.0
  800.0    0.0  104.0   164.0   625.0   194.0   274.0   325.0   481.0   349.0
  424.0  104.0    0.0   148.0   389.0   450.0   458.0   169.0   149.0   281.0
 1044.0  164.0  148.0     0.0   169.0   178.0   850.0   629.0   505.0   793.0
 1225.0  625.0  389.0   169.0     0.0   629.0  1629.0  1000.0   576.0  1314.0
 1714.0  194.0  450.0   178.0   629.0     0.0   788.0  1009.0  1109.0  1061.0
  754.0  274.0  458.0   850.0  1629.0   788.0     0.0   289.0   765.0   153.0
  125.0  325.0  169.0   629.0  1000.0  1009.0   289.0     0.0   136.0    34.0
  121.0  481.0  149.0   505.0   576.0  1109.0   765.0   136.0     0.0   306.0
  229.0  349.0  281.0   793.0  1314.0  1061.0   153.0    34.0   306.0     0.0

In [41]:
m = Model()
@variable(m, x[1:10,1:10] >= 0)
for i in 1:10
    @constraint(m, sum(x[:,i]) - sum(x[i,:]) + present[i] == required[i])
end
@objective(m,Min, sum(distances .* x))
solve(m)

:Optimal

### Optimal movement is

In [47]:
x_val = getvalue(x)

for i in 1:10
    for j in 1:10
        val = x_val[i,j]
        if val > 0
            println("Agency ", i , " sends " ,  x_val[i,j] , " cars to agency ", j )
        end
    end
end

Agency 2 sends 1.0 cars to agency 3
Agency 2 sends 5.0 cars to agency 6
Agency 2 sends 1.0 cars to agency 7
Agency 5 sends 3.0 cars to agency 4
Agency 8 sends 5.0 cars to agency 10
Agency 9 sends 2.0 cars to agency 1
Agency 9 sends 3.0 cars to agency 3
Agency 9 sends 1.0 cars to agency 8


# Question 3